In [384]:
import os
import cv2
import json
import numpy as np
import urllib.request

def load_json_from_scalabel():
    scalabel_data = []
    scalabel_path = anno_path + 'scalabel' # path to directory containing .json files
    for json_file in sorted(os.listdir(scalabel_path)):
        with open(os.path.join(scalabel_path, json_file)) as f:
            data = json.load(f)
            scalabel_data.extend(data)
    return scalabel_data

[{'name': 'https://s3.amazonaws.com/mc-imt/vehicle/2019X8248/detail_damage2/16609/medium/7B95777D-BDAB-44B2-9F46-196C74C40EF7.jpeg',
  'url': 'https://s3.amazonaws.com/mc-imt/vehicle/2019X8248/detail_damage2/16609/medium/7B95777D-BDAB-44B2-9F46-196C74C40EF7.jpeg',
  'videoName': '',
  'attributes': None,
  'timestamp': 10000,
  'index': 0,
  'labels': [{'id': 1,
    'category': 'headlight',
    'attributes': {'Big': False,
     'Damaged': True,
     'Front - Rear': [1, 'F'],
     'Left - Right': [1, 'L'],
     'Thin': True},
    'manual': True,
    'box2d': None,
    'poly2d': [{'vertices': [[101.08108108108108, 0],
       [53.513513513513516, 49.00900900900901],
       [52.072072072072075, 99.45945945945947],
       [89.54954954954955, 123.96396396396396],
       [174.5945945945946, 134.05405405405406],
       [305.76576576576576, 135.4954954954955],
       [441.26126126126127, 135.4954954954955],
       [595.4954954954956, 135.4954954954955],
       [598.3783783783784, 0]],
      'ty

In [385]:
def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [386]:
def download_images_from_s3():
    scalabel_data = load_json_from_scalabel()
    for index, img in enumerate(scalabel_data):
        image = url_to_image(img['name'])
        image_name = img['name'].replace('/', '_')
        cv2.imwrite(image_name, image)
    print(index, len(scalabel_data))

In [387]:
def extract_params_from_scalabel(img):
    points = []
    img_path = '' 
    try:
        for anno in img['labels']:
            global h,w
            poly2d_data = anno['poly2d']
            for p in poly2d_data:
                points.append(p['vertices'])
            image_name = img['name'].replace('/', '_')
            img = cv2.imread(img_path + image_name)
            h, w = img.shape[:2] # This is imageHeight, imageWidth
    except:
        pass
    return h, w

(340, 700)

In [388]:
def getLabelAndPoints(img):
    label = []
    points = []
    try:
        for i in img['labels']:
            label.append(i['category'])
            poly2d_data = i['poly2d']
            for p in poly2d_data:
                points.append(p['vertices'])
    except:
        pass
    return label, points

In [390]:
def convert_scalabel_to_dataturks():
    scalabel_data = load_json_from_scalabel()
    for _, img in enumerate(scalabel_data):
        height, width = extract_params_from_scalabel(img)
        label, points = getLabelAndPoints(img)
        name = img['name']
        anno = []
        labels = {}
        for i in range(len(label)):
            labels = {'label':[label[i]],'shape':'','points':points[i]}
            anno.append(labels)
        labels.update({'notes':'','imageWidth':width,'imageHeight':height})
        dataturk = {}
        dataturk['content'] = name
        dataturk['annotation'] = anno
        metadata = {'first_done_at':'','last_updated_at':'','sec_taken':'','last_updated_by':'','status':'','evaluation':''}
        dataturk.update({'extras': '','metadata': metadata})
        print(datat

{'content': 'https://s3.amazonaws.com/mc-imt/vehicle/2019X8248/detail_damage2/16609/medium/7B95777D-BDAB-44B2-9F46-196C74C40EF7.jpeg', 'annotation': [{'label': ['headlight'], 'shape': '', 'points': [[101.08108108108108, 0], [53.513513513513516, 49.00900900900901], [52.072072072072075, 99.45945945945947], [89.54954954954955, 123.96396396396396], [174.5945945945946, 134.05405405405406], [305.76576576576576, 135.4954954954955], [441.26126126126127, 135.4954954954955], [595.4954954954956, 135.4954954954955], [598.3783783783784, 0]]}, {'label': ['crack'], 'shape': '', 'points': [[278.3783783783784, 8.64864864864865], [327.3873873873874, 63.42342342342342], [398.01801801801804, 103.78378378378379], [461.44144144144144, 105.22522522522523], [545.0450450450451, 87.92792792792793], [589.7297297297298, 66.30630630630631], [589.7297297297298, 8.64864864864865]]}, {'label': ['crack'], 'shape': '', 'points': [[406.9892473118279, 416.1290322580645], [422.04301075268813, 433.3333333333333], [429.5698

{'content': 'https://s3.amazonaws.com/mc-imt/vehicle/2019X8121/detail_damage1/16557/medium/image.jpg', 'annotation': [{'label': ['fender'], 'shape': '', 'points': [[0.1801801801801802, 24.504504504504506], [595.4954954954956, 27.38738738738739], [598.3783783783784, 327.2072072072072], [520.5405405405405, 347.3873873873874], [468.64864864864865, 394.954954954955], [438.3783783783784, 480], [412.43243243243245, 580.9009009009009], [203.42342342342343, 580.9009009009009], [178.91891891891893, 562.1621621621622], [165.94594594594597, 507.3873873873874], [145.76576576576576, 497.2972972972973], [99.63963963963964, 490.0900900900901], [67.92792792792793, 481.44144144144144], [46.30630630630631, 429.54954954954957], [160.18018018018017, 439.63963963963965], [164.5045045045045, 429.54954954954957], [121.26126126126127, 230.63063063063063], [103.96396396396396, 204.6846846846847], [76.57657657657658, 188.82882882882885], [0.1801801801801802, 160]]}, {'label': ['dent'], 'shape': '', 'points': [[

IndexError: list index out of range